# Test Article

Загружаем библиотеки

In [1]:
from transformers import AutoTokenizer, AutoModel, pipeline

In [2]:
modelpath = "../model/fine-train"
# modelpath = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModel.from_pretrained(modelpath)

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at ../model/fine-train and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
text = "После начала российской военной спецоперации на Украине западные страны усилили санкционное давление на Москву. МИД России призвал стороны прекратить боевые действия."

In [6]:
import re

tr = 0.001

def mask_bert_sent(text, model, tokenizer):
    unmasker = pipeline("fill-mask", model=model, tokenizer=tokenizer)
    p = re.compile(r'[\w-]+')
    iter = p.finditer(text)
    listErr = []
    for match in iter:
        if match.start() == 0:
            masktext = "[MASK]" + text[match.end():]
        elif match.end() == len(text):
            masktext = text[:match.start()] + "[MASK]"
        else:
            masktext = text[:match.start()] + "[MASK]" + text[match.end():]
        
        res = unmasker(masktext, targets=[match.group()])
        if res[0]['score'] < tr:
            errorrDesc = {
                "word": match.group(),
                "start": match.start(),
                "end": match.end(),
                "prob": res[0]['score']
            }
            listErr.append(errorrDesc)
    return listErr


In [7]:
errors = mask_bert_sent(text, modelpath, tokenizer)
if len(errors) > 0:
    for e in errors:
        print (e)

TypeError: FillMaskPipeline._sanitize_parameters() got an unexpected keyword argument 'tokenizer'